In [1]:
print("Hello")

Hello


In [4]:
import pandas as pd

df = pd.read_csv(r"..\data\icici\result.csv")

df.shape

(100, 5)

In [5]:
EXPECTED_COLS = ["Date", "Description", "Debit Amt", "Credit Amt", "Balance"]

def parse(pdf_path: str) -> pd.DataFrame:
    try:
        import camelot  # type: ignore
    except Exception as e:
        raise ImportError("Camelot not installed or failed to import") from e

    tables = camelot.read_pdf(pdf_path, pages="all", flavor="stream")
    if not tables:
        raise ValueError("No tables detected in PDF; try different flavor or pdfplumber fallback")

    frames = [t.df for t in tables if not t.df.empty]
    if not frames:
        raise ValueError("Extracted tables are empty")
    raw = pd.concat(frames, ignore_index=True)

    raw = raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)

    # Use first row as header if all string-like; then remove any repeated header rows appearing later
    header_tokens: list[str] | None = None
    if raw.shape[0] > 1 and all(isinstance(x, str) and x.strip() for x in raw.iloc[0].tolist()):
        header_tokens = [str(c).strip() for c in raw.iloc[0].tolist()]
        raw.columns = header_tokens
        raw = raw.iloc[1:].reset_index(drop=True)

    if header_tokens:
        # Drop subsequent rows that exactly repeat the header (multi-page repeated headers)
        duplicate_header_mask = raw.apply(
            lambda r: [("" if v is None else str(v).strip()) for v in r.tolist()] == header_tokens,
            axis=1,
        )
        if duplicate_header_mask.any():
            raw = raw.loc[~duplicate_header_mask].reset_index(drop=True)

    aliases = {
        "Txn Date": "Date",
        "Transaction Date": "Date",
        "Narration": "Description",
        "Withdrawal Amount": "Debit Amt",
        "Withdrawal": "Debit Amt",
        "Debit": "Debit Amt",
        "Deposit Amount": "Credit Amt",
        "Deposit": "Credit Amt",
        "Credit": "Credit Amt",
        "Closing Balance": "Balance",
        "Balance Amount": "Balance",
    }

    rename_map: dict[str, str] = {}
    for c in raw.columns:
        tgt = aliases.get(c)
        if tgt in EXPECTED_COLS:
            rename_map[c] = tgt
    df = raw.rename(columns=rename_map)

    for col in EXPECTED_COLS:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[EXPECTED_COLS]

    if "Date" in df.columns:
        try:
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)
        except Exception:
            pass

    numeric_cols = [c for c in ["Debit Amt", "Credit Amt", "Amount", "Balance"] if c in df.columns]
    for col in numeric_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.replace("₹", "", regex=False)
            .str.replace("INR", "", regex=False)
            .str.replace("Dr", "", regex=False)
            .str.replace("Cr", "", regex=False)
            .str.strip()
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


parse(r"..\data\icici\icici sample.pdf")

e:\work\Projects\ai-agent-challenge\.venv\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
C:\Users\MB511WS\AppData\Local\Temp\ipykernel_24120\776352599.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Date,Description,Debit Amt,Credit Amt,Balance
0,2024-08-01,Salary Credit XYZ Pvt Ltd,1935.30,NaN,6864.58
1,2024-08-02,Salary Credit XYZ Pvt Ltd,NaN,1652.61,8517.19
2,2024-08-03,IMPS UPI Payment Amazon,3886.08,NaN,4631.11
3,2024-08-03,Mobile Recharge Via UPI,NaN,1648.72,6279.83
4,2024-08-14,Fuel Purchase Debit Card,NaN,3878.57,10158.40
...,...,...,...,...,...
95,2025-07-20,Utility Bill Payment Electricity,NaN,2989.23,3926.14
96,2025-07-23,Salary Credit XYZ Pvt Ltd,NaN,2988.46,6914.60
97,2025-07-24,Electricity Bill NEFT Online,2917.52,NaN,3997.08
98,2025-07-25,Salary Credit XYZ Pvt Ltd,566.32,NaN,3430.76
